In [1]:

import ibmos2spark

 # @hidden_cell
credentials = {
    'auth_url': 'https://identity.open.softlayer.com',
    'project_id': '8aa6e0aab6094fdd9838d3e08e7253f3',
    'region': 'dallas',
    'user_id': '775a8781e7bf4c0b88ebc086525aa3d7',
    'username': 'member_78415bb429eb12ae25686a6d27742576ced7a7ca',
    'password': 'IJ3.9-K=12.]s&r?'
}

configuration_name = 'os_fa1cd21c20344b08a7e27d96dc1f7387_configs'
bmos = ibmos2spark.bluemix(sc, credentials, configuration_name)

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
# Please read the documentation of PySpark to learn more about the possibilities to load data files.
# PySpark documentation: https://spark.apache.org/docs/2.0.1/api/python/pyspark.sql.html#pyspark.sql.SparkSession
# The SparkSession object is already initalized for you.
# The following variable contains the path to your file on your Object Storage.



In [2]:
path_user = bmos.url('Fall2017', 'user.json.gz')
df_users=spark.read.json(path_user)
df_users.show(5)
df_users.printSchema()

+-------------+---------------+---------------+----------------+--------------+---------------+---------------+---------------+-----------------+----------------+------------------+-----------------+-----+--------------------+----+--------------------+-----+------+------------+------+--------------------+-------------+
|average_stars|compliment_cool|compliment_cute|compliment_funny|compliment_hot|compliment_list|compliment_more|compliment_note|compliment_photos|compliment_plain|compliment_profile|compliment_writer| cool|               elite|fans|             friends|funny|  name|review_count|useful|             user_id|yelping_since|
+-------------+---------------+---------------+----------------+--------------+---------------+---------------+---------------+-----------------+----------------+------------------+-----------------+-----+--------------------+----+--------------------+-----+------+------------+------+--------------------+-------------+
|          3.8|           5174|      

In [3]:
print"Yelp user record count:",df_users.count()

Yelp user record count: 1183362


In [4]:
df_users.describe('funny','useful','cool').show()

+-------+------------------+------------------+------------------+
|summary|             funny|            useful|              cool|
+-------+------------------+------------------+------------------+
|  count|           1183362|           1183362|           1183362|
|   mean|15.256665331487744|30.294972290812108|20.322260643826656|
| stddev| 490.7658665490963| 705.8451812603226|  668.138310221103|
|    min|                 0|                 0|                 0|
|    max|            192663|            202867|            201062|
+-------+------------------+------------------+------------------+



In [5]:
df_users.select('user_id','name','funny','useful','cool').sort('cool',ascending=False).show(10)

+--------------------+-------+------+------+------+
|             user_id|   name| funny|useful|  cool|
+--------------------+-------+------+------+------+
|JjXuiru1_ONzDkYVr...|Richard|192663|202867|201062|
|0XXIv9a0LWiaCjAku...|  David| 20437| 30843|195341|
|PhUqhfyk3jdaS0Xb6...|    Des| 24289|166693|175230|
|6s-g2vFu12OemhiK3...|   Dave| 84527|187179|167181|
|ax7SnXOTIpatbsmqH...| Rohlin|131408|132117|131781|
|--2vR0DIsmQ6WfcSz...| Harald|122419|122921|122890|
|WR6033peFiviFPzAV...|   Dale| 63623|150844|104577|
|PvSzDcag7vo1e5Zz7...|  Chris|103514|104599|103668|
|NOUfyJW-BAo_-Cbfo...|  Lolia| 25715| 97256| 96424|
|XCW5pJCDhvlW76XTS...|    Ivy| 79316| 80151| 79437|
+--------------------+-------+------+------+------+
only showing top 10 rows



In [6]:
df_users.corr('funny','compliment_funny')

0.6871378854374348

In [8]:
df_users.createOrReplaceTempView("yelp_users")
spark.sql("SELECT YEAR(yelping_since) AS yelping_year,COUNT(*) AS user_count FROM yelp_users GROUP BY YEAR(yelping_since)ORDER BY YEAR(yelping_since)").show()

+------------+----------+
|yelping_year|user_count|
+------------+----------+
|        2004|        73|
|        2005|       959|
|        2006|      5812|
|        2007|     15950|
|        2008|     31284|
|        2009|     58336|
|        2010|     94826|
|        2011|    142915|
|        2012|    150292|
|        2013|    165652|
|        2014|    181419|
|        2015|    173071|
|        2016|    121251|
|        2017|     41522|
+------------+----------+



In [15]:
from pyspark.sql.types import*
ssaSchema=StructType([\
StructField("ssa_year",IntegerType(),True),\
StructField("ssa_name",StringType(),True),\
StructField("ssa_gender",StringType(),True),\
StructField("ssa_count",IntegerType(),True)])
df_gender = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .schema(ssaSchema)\
  .load(bmos.url('Fall2017', 'ssa_names_gender.csv'))
print"SSA record count:",df_gender.count()
df_gender.take(5)
df_gender.printSchema()


SSA record count: 1891894
root
 |-- ssa_year: integer (nullable = true)
 |-- ssa_name: string (nullable = true)
 |-- ssa_gender: string (nullable = true)
 |-- ssa_count: integer (nullable = true)



In [17]:
df_gender.createOrReplaceTempView("gender")
spark.sql("SELECT ssa_name,SUM(ssa_count) AS total FROM gender WHERE ssa_gender='F'GROUP BY ssa_name ORDER BY SUM(ssa_count)DESC").show()

+---------+-------+
| ssa_name|  total|
+---------+-------+
|     Mary|4120692|
|Elizabeth|1620611|
| Patricia|1571380|
| Jennifer|1465243|
|    Linda|1451809|
|  Barbara|1433727|
| Margaret|1244388|
|    Susan|1121148|
|  Dorothy|1106575|
|    Sarah|1069797|
|  Jessica|1043625|
|    Helen|1017519|
|    Nancy|1001671|
|    Betty| 999275|
|    Karen| 985097|
|     Lisa| 964673|
|     Anna| 883882|
|   Sandra| 873187|
|   Ashley| 841259|
| Kimberly| 833311|
+---------+-------+
only showing top 20 rows



In [18]:
spark.sql("SELECT ssa_gender,COUNT(DISTINCT ssa_name) AS name_count FROM gender GROUP BY ssa_gender").show()

+----------+----------+
|ssa_gender|name_count|
+----------+----------+
|         F|     66358|
|         M|     40337|
+----------+----------+



In [19]:
spark.sql("SELECT COUNT(DISTINCT ssa_name) AS name_cnt FROM gender").show()

+--------+
|name_cnt|
+--------+
|   96174|
+--------+



In [34]:
df_women=spark.sql("SELECT ssa_name, SUM(ssa_count)AS women_count, 0 AS men_count FROM gender WHERE ssa_gender='F' GROUP BY ssa_name")
df_men=spark.sql("SELECT ssa_name, 0 AS women_count, SUM(ssa_count) AS men_count FROM gender WHERE ssa_gender='M' GROUP BY ssa_name")
df_women.unionAll(df_men).createOrReplaceTempView("women_men_union")
spark.sql("SELECT ssa_name, SUM(women_count)AS women, SUM(men_count) AS men FROM women_men_union GROUP BY ssa_name").createOrReplaceTempView("women_men") 
df_gender_ratio=spark.sql("SELECT ssa_name, women, men, IF(men>women, 2, 1) AS gender, IF(men>women, men/(women+men),women/(women+men))AS genderRatio FROM women_men")
df_gender_ratio.show()

+--------+-----+------+------+------------------+
|ssa_name|women|   men|gender|       genderRatio|
+--------+-----+------+------+------------------+
|   Abner|    0|  7440|     2|               1.0|
|Laurence|  454| 39105|     2|0.9885234712707601|
| Ephriam|    0|   921|     2|               1.0|
|Thurston|    0|  3413|     2|               1.0|
|   Tyler|16245|580385|     2|0.9727720697919984|
| Rosendo|    0|  5179|     2|               1.0|
|   Ewart|    0|   131|     2|               1.0|
|    Nell|26638|    98|     1|0.9963345302214243|
|    Faye|67691|   518|     1|0.9924056942632205|
|   Lorne|   50|  4235|     2|0.9883313885647608|
| Percell|    0|  1064|     2|               1.0|
|   Fines|    0|    81|     2|               1.0|
| Normand|    0|  5829|     2|               1.0|
|Francois|    5|  1677|     2|0.9970273483947681|
|    Mirl|   55|    87|     2|0.6126760563380281|
|  Helmut|    0|   451|     2|               1.0|
|  Clance|    0|   336|     2|               1.0|


In [38]:
df_users.select(df_users.user_id, df_users.name).createOrReplaceTempView("yelp_users")
df_gender_ratio.createOrReplaceTempView("gender_ratio")
df_user_gender=spark.sql("SELECT U.user_id, U.name, IF(G.gender IS NULL, 3, G.gender) AS user_gender, IF(G.gender IS NULL, 1.00, G.genderRatio) AS gender_ratio FROM yelp_users AS U LEFT JOIN gender_ratio AS G ON U.name=G.ssa_name")
df_user_gender.createOrReplaceTempView("user_gender_ratio")
df_user_gender.show(10)

+--------------------+------------------+-----------+------------+
|             user_id|              name|user_gender|gender_ratio|
+--------------------+------------------+-----------+------------+
|KVAMAb7X6gf-FTjpv...|         2 S M Art|          3|         1.0|
|jQq-S5CIiRrgGBXw2...|86 The Bad Reviews|          3|         1.0|
|jXsHj5vXj_fNbQceD...|       A. Brigitte|          3|         1.0|
|VCNdLIFlwa4buWYIV...|               A.J|          3|         1.0|
|E3LyTOlYe0KmB25bq...|               A.J|          3|         1.0|
|7Ni48EZUbWatCYo7g...|               A.J|          3|         1.0|
|Gy5L7iJSN0uExQKiv...|               A.J|          3|         1.0|
|2LPRuLyo5RRmhGeJ7...|              A.a.|          3|         1.0|
|57NU6K2E1jsWcVf13...|              ANNA|          3|         1.0|
|W34pNbx4TRDoZ40aL...|              ANNA|          3|         1.0|
+--------------------+------------------+-----------+------------+
only showing top 10 rows



In [39]:
spark.sql("SELECT COUNT(*) FROM user_gender_ratio WHERE user_gender=3").show()

+--------+
|count(1)|
+--------+
|  155836|
+--------+



In [41]:
df_user_gender=spark.sql("SELECT U.user_id, U.name, IF(G.gender IS NULL, 3, G.gender) AS gender_ratio, IF(G.gender IS NULL, 1.00,G.genderRatio)AS gender_ratio FROM yelp_users AS U LEFT JOIN gender_ratio AS G ON LOWER(U.name)=LOWER(G.ssa_name)")
spark.sql("SELECT COUNT(*) FROM user_gender_ratio WHERE user_gender=3").show()

+--------+
|count(1)|
+--------+
|  155836|
+--------+



In [42]:
spark.sql("SELECT COUNT(*) FROM user_gender_ratio WHERE user_gender=3").show()

+--------+
|count(1)|
+--------+
|  155836|
+--------+



In [46]:
df_user_gender=spark.sql("SELECT U.user_id, U.name, IF(G.gender IS NULL, 3, G.gender) AS gender_ratio, IF(G.gender IS NULL, 1.00,G.genderRatio)AS gender_ratio FROM yelp_users AS U LEFT JOIN gender_ratio AS G ON LOWER(U.name)=LOWER(G.ssa_name)")
df_user_gender.show()

+--------------------+---------------+------------+------------------+
|             user_id|           name|gender_ratio|      gender_ratio|
+--------------------+---------------+------------+------------------+
|8yuMP766uRPe6sEcx...|         'Nikki|           3|               1.0|
|MS_IRLwpy-TUGNyUI...|1Southerngyrl76|           3|               1.0|
|k7aF28JCVxi14O4qO...|         A-dogg|           3|               1.0|
|WwsMmD9R4Oz0ZqmsY...|       Advice4u|           3|               1.0|
|Zu7gGW6dlGIf3iDG9...|       Adwinnie|           3|               1.0|
|D809l3fCdGAKv897u...|       Agnes C.|           3|               1.0|
|qMubZnWM_lLRXqYwE...|      Agustinus|           3|               1.0|
|pivG5Kp8WdoCekuM6...|          Ailis|           1|               1.0|
|hJPPMAaczO7VgE-sC...|          Ailis|           1|               1.0|
|CiK2bpgH8G35aofeo...|       Air Rick|           3|               1.0|
|r_yzcbcHGOgtmGKRC...|          Akeem|           2|               1.0|
|YDwKu

In [50]:
df_user_gender=spark.sql("SELECT U.user_id, U.name, IF(G.gender IS NULL, 3, G.gender) AS gender_ratio, IF(G.gender IS NULL, 1.00,G.genderRatio)AS gender_ratio FROM yelp_users AS U LEFT JOIN gender_ratio AS G ON LOWER(U.name)=LOWER(G.ssa_name)")
spark.sql("SELECT COUNT(*)FROM user_gender_ratio WHERE user_gender=3").show()

+--------+
|count(1)|
+--------+
|  155836|
+--------+



In [53]:
df_users.select(df_users.user_id, df_users.name).createOrReplaceTempView("yelp_users")
df_gender_ratio.createOrReplaceTempView("gender_ratio")
df_user_gender=spark.sql("SELECT U.user_id, U.name, IF(G.gender IS NULL, 3, G.gender) AS user_gender, IF(G.gender IS NULL, 1.00, G.genderRatio) AS gender_ratio FROM yelp_users AS U LEFT JOIN gender_ratio AS G ON LOWER(U.name)=LOWER(G.ssa_name)")
df_user_gender.createOrReplaceTempView("user_gender_ratio")
spark.sql("SELECT COUNT(*) FROM user_gender_ratio WHERE user_gender=3").show()


+--------+
|count(1)|
+--------+
|  133373|
+--------+

